In [1]:
from util import *
from glob import glob
import matplotlib.pyplot as plt
from shapely import wkt
pd.set_option("display.max_columns", None)

In [2]:
gdf = gpd.read_file(f"Manuwatu_Test/transects_intersects_20240315_120828.geojson")
gdf["Date"] = pd.to_datetime(gdf.ShorelineID, dayfirst=True, format='mixed')
gdf["Year"] = gdf.Date.dt.year
gdf["YearsSinceBase"] = (gdf.Date - pd.Timestamp(1800, 1, 1)).dt.days / 365.25
gdf["YearsUntilFuture"] = (
    pd.Timestamp(2100, 1, 1) - gdf.Date
    ).dt.days / 365.25
gdf.Date = gdf.Date.astype(str)
gdf

,TransectID,ShorelineID,BaselineID,Distance,IntersectX,IntersectY,Uncertainty,geometry,Date,Year,YearsSinceBase,YearsUntilFuture
0,1,05/12/2020,0,-30.72,1.664823e+06,5.652036e+06,10.00,POINT (1664822.752 5652035.621),2020-12-05,2020,220.922656,79.071869
1,1,11/02/2017,0,-29.69,1.664822e+06,5.652035e+06,10.00,POINT (1664821.739 5652035.474),2017-02-11,2017,217.108830,82.885695
2,1,08/11/1970,0,-27.86,1.664820e+06,5.652035e+06,2.63,POINT (1664819.922 5652035.211),1970-11-08,1970,170.847365,129.147159
3,1,19/09/1955,0,-25.63,1.664818e+06,5.652035e+06,2.68,POINT (1664817.715 5652034.890),1955-09-19,1955,155.709788,144.284736
4,2,05/12/2020,0,-31.94,1.664830e+06,5.651936e+06,10.00,POINT (1664830.066 5651936.243),2020-12-05,2020,220.922656,79.071869
...,...,...,...,...,...,...,...,...,...,...,...,...
9214,1925,23/01/2016,244,-39.52,1.904293e+06,5.511354e+06,1.02,POINT (1904293.154 5511353.967),2016-01-23,2016,216.054757,83.939767
9215,1925,31/03/1944,244,-29.47,1.904294e+06,5.511344e+06,2.27,POINT (1904293.505 5511343.923),1944-03-31,1944,144.240931,155.753593
9216,1926,29/04/2021,244,-36.22,1.904393e+06,5.511358e+06,1.02,POINT (1904392.891 5511358.332),2021-04-29,2021,221.319644,78.674880
9217,1926,23/01/2016,244,-36.58,1.904393e+06,5.511359e+06,1.02,POINT (1904392.878 5511358.698),2016-01-23,2016,216.054757,83.939767


In [ ]:
lines = gpd.read_file("Manuwatu_Test/t10_transects_azi.shp")
lines

In [ ]:
def get_azimuth(line):
    p1, p2 = line.coords[1:]
    azimuth = math.degrees(math.atan2(p2[0]-p1[0], p2[1]-p1[1]))
    if azimuth < 0:
        azimuth += 360
    return azimuth
lines["calculated_azimuth"] = lines.geometry.apply(get_azimuth)
lines

In [ ]:
gdf.Year.describe()

In [ ]:
gdf.Distance.describe()

In [ ]:
#DSASv6 does not produce an azimuth attribute in the transect file so either need to calculate geometry in ArcGIS Pro or add code
transect_metadata = get_transect_metadata(f"Manuwatu_Test/t10_transects_azi.shp")

In [ ]:
linear_models = fit(gdf, transect_metadata)
rolled_slopes = linear_models.groupby("group").slope.rolling(10, min_periods=1).mean().dropna().reset_index(level=0).win_type='gaussian'
linear_models.slope = rolled_slopes.slope
linear_models.dropna(inplace=True)
linear_models

In [ ]:
#Changed coordinate function by making old_x and old_y negative 
def calculate_new_coordinates(old_x, old_y, bearing, distance):
    bearing_radians = math.radians(bearing)
    new_x = old_x + (distance * math.sin(bearing_radians))
    new_y = old_y + (distance * math.cos(bearing_radians))
    point = Point(new_x, new_y)
    assert not point.is_empty
    return point

#Removed other model equations and changed Azimuth addtion from 180 to 360 deg
def predict(
    df: pd.DataFrame,
    linear_models: pd.DataFrame,
    transect_metadata: dict,
):
    """_summary_

    Args:
        df (pd.DataFrame): dataframe with columns: TransectID, Date, Distance, YearsSinceBase
        linear_models (pd.DataFrame): dataframe with columns: TransectID, slope, intercept
        transect_metadata (dict): dict lookup of TransectID to Azimuth & group
        
    Returns:
        pd.DataFrame: resulting prediction points for the year 2100
    """
    results = []
    for i, row in linear_models.iterrows():
        transect_ID = row.TransectID
        transect_df = df[df.TransectID == transect_ID]
        latest_row = transect_df[transect_df.Date == transect_df["Date"].max()].iloc[0]
        future_year = int(row.get("FUTURE_YEAR", FUTURE_YEAR))
        result = row.to_dict()
        result.update({
            "TransectID": transect_ID,
            "BaselineID": latest_row.BaselineID,
            "group": row.group,
            "Year": future_year,
            "ocean_point": calculate_new_coordinates(
                latest_row.geometry.x,
                latest_row.geometry.y,
                transect_metadata[transect_ID]["Azimuth"] + 180,
                500,
            ),
        })
        
        model = "linear"
        slope = row.slope
        intercept = row.intercept

        predicted_distance = slope * (future_year - 1800) + intercept
        distance_difference = latest_row.Distance - predicted_distance
        result[f"{model}_model_point"] = calculate_new_coordinates(
            latest_row.geometry.x,
            latest_row.geometry.y,
            transect_metadata[transect_ID]["Azimuth"],
            distance_difference,
        )
        result[f"{model}_model_predicted_distance"] = predicted_distance
        result[f"{model}_model_distance"] = distance_difference
        results.append(result)
    results = gpd.GeoDataFrame(results)
    return results

In [ ]:
results = predict(gdf, linear_models, transect_metadata)
results

In [ ]:
results.set_geometry("linear_model_point", inplace=True, crs=2193)
results

In [ ]:
poly = prediction_results_to_polygon(results)
output_shapefile = "Manuwatu_Test/weighted_projection_output.shp"
poly.to_file(output_shapefile, driver="ESRI Shapefile")

In [ ]:
m = poly.explore(tiles="Esri.WorldImagery")
gpd.GeoDataFrame(results.drop(columns=["ocean_point", "linear_model_point"]), geometry=results.linear_model_point).explore(m=m)
gdf.explore("Year", legend=True, m=m)